Data & kaggle_note: https://www.kaggle.com/harlfoxem/housesalesprediction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing DataSet 
dataset = pd.read_csv("../input/kc-housesales-data/kc_house_data.csv")
#Keggle note 우측 데이터 "kc_house_data.csv" click 후 path 복사

# 데이터 파악하기 

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset.dtypes

In [ ]:
dataset.describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset['bedrooms'].value_counts()

# EDA(Exploratory Data Analysis)

In [ ]:
fig,axes=plt.subplots(nrows=1,ncols=1,figsize=(15,10))
sns.countplot(dataset.bedrooms,order=dataset['bedrooms'].value_counts().index)

In [ ]:
fig,axes=plt.subplots(nrows=1,ncols=1,figsize=(15,10))
sns.distplot(dataset['price'],hist=True,kde=True,rug=False,label='price',norm_hist=True)

In [ ]:
variables = dataset[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 
       'lat', 'long', 'sqft_living15', 'sqft_lot15']]

In [ ]:
colormap = plt.cm.PuBu 
plt.figure(figsize=(15, 10)) 
plt.title("Person Correlation of Features", y = 1.05, size = 15) 
sns.heatmap(variables.astype(float).corr(), linecolor = "white", cmap = colormap, annot = True)
            #, linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, 
         #   linecolor = "white", annot = True, annot_kws = {"size" : 16})


 - 독립 변수간 상관 관계를 보이는 것을 다중공선성(Multicollinearity)이라 하며, 다중공선성이 있으면 부정확한 회귀 결과가 도출됨
 - X는 Y와 상관관계가 높아야 하고 독립변수끼리 상관관계가 없어야 비교적 정확한 결과를 얻을 수 있음
 -- 계수 추정이 잘 되지 않거나 불안정해져서 데이터가 약간만 바뀌어도 추정치가 크게 달라질 수 있음
 -- 계수가 통계적으로 유의미하지 않은 것처럼 나올 수 있음

# 다중공선성 변수 제거

In [ ]:
# bedroom이 종속변수인 price에 영향을 미치는 관계를 구하기 위한 회귀분석 

dataset['intercept'] = 1
pm = sm.OLS(dataset['price'], dataset[['intercept','bedrooms']])
results_p = pm.fit()
results_p.summary()

 - bedroom이 많을 수록 price가 비싸다는 양의 회귀계수가 도출 됨(다른 변수의 영향을 받지 않았을 때)

In [ ]:
#'sqft_living'-'sqft_above', 
dataset['intercept'] = 1
lm = sm.OLS(dataset['price'], dataset[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 
       'lat', 'long', 'sqft_living15', 'sqft_lot15']])
results = lm.fit()
results.summary()

 - 변수가 많아질수록 다중공선성의 발생으로 회귀결과가 잘못 도출 될 수 있음
 - bedrooms coef = -3.549e+04 : bedroom이 많을 수록 price가 떨어진다는 이상한 결과가 도출됨
 - 이처럼 다중공선성이 있다면 회귀 결과를 왜곡하게 됨, 따라서 서로 상관관계가 높은 변수들 중 하나를 제거해야 함


In [ ]:
#, 'bathrooms', 'sqft_living', 'sqft_above', 'sqft_living15'  변수 제거
dataset['intercept'] = 1
lm_re = sm.OLS(dataset['price'], dataset[['bedrooms', 
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
        'sqft_basement', 'yr_built', 'yr_renovated', 
       'lat', 'long','sqft_lot15']])
results_re = lm_re.fit()
results_re.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X = dataset[['bedrooms', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
        'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long','sqft_lot15']]
y = dataset['price']

In [ ]:
# Train, Test set 80:20 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# 다중 선형회귀모델 학습 및 예측

In [ ]:
# 회귀모델 학습
model =  LinearRegression()
model.fit(X_train, y_train)
print(model.coef_)
print(model.intercept_)

y(price) = -13543932.428068105 + 2.43119521e+04X1 + 1.99257992e-01X2 + 5.95084840e+04X3 + .... + 1.59292252e-02X13

In [ ]:
# X_test set 에 대한 예측 결과
y_pred = model.predict(X_test)
y_pred

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

#실제 y값과 예측 값 비교
y_compare={'y_test':y_test, 'y_predicted':y_pred}
pd.DataFrame(y_compare)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print("mean_squared_error: ", mean_squared_error(y_test, y_pred))
print("mean_root_squared_error: ", mean_squared_error(y_test, y_pred,squared=False))
print("mean_absolute_error: ", mean_absolute_error(y_test, y_pred))